In [ ]:
import requests
import hashlib
import time
import json

In [ ]:
API_KEY = "4YCF385ZNXLEHADRUSFV"
API_SECRET = "YcjAfd9x7S$$qX^4u#n$rvLD$X6c^pT87ShWvF3V"

In [ ]:
# Generate the timestamp
timestamp = str(int(time.time()))
auth_hash = hashlib.sha1((API_KEY + API_SECRET + timestamp).encode('utf-8')).hexdigest()

headers = {
    "X-Auth-Date": timestamp,
    "X-Auth-Key": API_KEY,
    "Authorization": auth_hash,
    "User-Agent": "PodcastIndex-Client"
}

# Step 1: Get Trending Podcasts
url = "https://api.podcastindex.org/api/1.0/podcasts/trending"
params = {"max": 200}  # Get up to 200 trending podcasts

response = requests.get(url, headers=headers, params=params)
trending_podcasts = response.json().get("feeds", [])

print(f"Retrieved {len(trending_podcasts)} trending podcasts.")

In [ ]:

# Step 1: Download trending podcasts
url = "https://api.podcastindex.org/api/1.0/podcasts/trending"

response = requests.get(url, headers=headers)
trending_data = response.json()

# Step 2: Extract relevant fields
trending_podcasts = trending_data.get("feeds", [])

simplified_podcasts = [
    {
        "title": feed.get("title"),
        "url": feed.get("url"),
        "description": feed.get("description"),
        "id": feed.get("id")
    }
    for feed in trending_podcasts
]

# Step 3: Save to JSON file
output_path = "top_podcasts.json"
with open(output_path, "w") as f:
    json.dump(simplified_podcasts, f, indent=4)

print(f"Saved {len(simplified_podcasts)} podcasts to {output_path}")


In [ ]:
# Step 2: Get Episodes for Each Podcast
all_episodes = []

for podcast in trending_podcasts:
    feed_id = podcast.get("id")
    podcast_title = podcast.get("title")

    # Fetch episodes for the podcast
    episodes_url = f"https://api.podcastindex.org/api/1.0/episodes/byfeedid"
    response = requests.get(episodes_url, headers=headers, params={"id": feed_id})

    if response.status_code == 200:
        episodes = response.json().get("items", [])
        for episode in episodes:
            audio_url = episode.get("enclosureUrl")
            episode_title = episode.get("title")

            # Collect relevant data
            all_episodes.append({
                "podcast": podcast_title,
                "episode_title": episode_title,
                "audio_url": audio_url
            })
    else:
        print(f"Failed to get episodes for feed ID {feed_id}")

print(f"Collected {len(all_episodes)} episodes.")

# Print sample data
print(all_episodes[:5])  # Print the first 5 entries



In [ ]:
import json
with open("podcast_data.json", "w") as json_file:
    json.dump(all_episodes, json_file, indent=4)

In [ ]:
# Iterate through each podcast entry
for episode in all_episodes:
    podcast_name = episode.get("podcast", "unknown_podcast")
    episode_title = episode.get("episode_title", "unknown_episode")
    audio_url = episode.get("audio_url")

    # Construct output filename safely
    safe_title = episode_title.replace(" ", "_").replace("/", "_")
    output_file = f"{podcast_name}_{safe_title}.txt"

    # Check if the audio_url is valid
    if audio_url and audio_url.strip():
        print(f"Transcribing {episode_title} from {audio_url}")
        try:
            # Run podscript command
            result = subprocess.run(
                [
                    "/Users/terezasaskova/go/bin/podscript",
                     "assembly-ai",
                     "--from-url", audio_url,
                    "--output", output_file
                ],
                check=True,
                capture_output=True,
                text=True
            )

            print(f"✅ Transcript saved to {output_file}")

        except subprocess.CalledProcessError as e:
            print(f"❌ Subprocess error:\n{e.stderr}")
        except Exception as e:
            print(f"❌ Exception occurred: {e}")
    else:
        print(f"❌ No valid audio URL found for {episode_title}")

print("Transcription process complete.")
